In [ ]:
from nemo.collections.tts.models import magpietts
from nemo.collections.tts.data.text_to_speech_dataset import MagpieTTSDataset, DatasetSample
from nemo.collections.tts.data.text_to_speech_dataset_lhotse import MagpieTTSLhotseDataset, setup_tokenizers
from omegaconf.omegaconf import OmegaConf, open_dict
import torch
import os
import soundfile as sf
from IPython.display import display, Audio
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

### Checkpoint Paths

In [ ]:
#checkpoint_file = "/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__LT_768_4_12/checkpoints/magpie-tts--val_loss=10.8018-epoch=87-last.ckpt"
#checkpoint_file = "/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__LT_768_4_12/checkpoints/magpie-tts--val_loss=10.6553-epoch=268-last.ckpt"
#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_2x_MG_768_4_12/checkpoints/magpie-tts--val_loss=10.6772-epoch=231-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_2x_MG_768_4_12/hparams.yaml"

#checkpoint_file = "/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_4x_MG_768_4_12/checkpoints/magpie-tts--val_loss=11.3472-epoch=198-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_4x_MG_768_4_12/hparams.yaml"
# checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_3x_AR_768_4_12/checkpoints/magpie-tts--val_loss=10.7700-epoch=136-last.ckpt" 
# hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_3x_AR_768_4_12/hparams.yaml"
#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_3x_MG_768_4_12/checkpoints/magpie-tts--val_loss=11.0669-epoch=200-last.ckpt"#
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_3x_MG_768_4_12/hparams.yaml"
#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_3x_AR_768_4_12/checkpoints/magpie-tts--val_loss=10.7198-epoch=238-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_3x_AR_768_4_12/hparams.yaml"

#checkpoint_file = "/datap/misc/continuouscheckpoints/maskgit/2x/dcce_1x_AR_768_4_12/checkpoints/magpie-tts--val_loss=9.8141-epoch=204-last.ckpt"#
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_1x_AR_768_4_12/hparams.yaml"

#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/lhotse_dcce_lt_AR_768_4_12/checkpoints/MagpieTTS-EN-Lhotse--val_loss=9.5592-step=231018-epoch=32-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/lhotse_dcce_lt_AR_768_4_12/hparams.yaml"

#checkpoint_file="/datap/misc/continuouscheckpoints/2505_release/grpo_lr2e-7_CerSsim55_ep1.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/2505_release/wandb_hparams.yaml"

#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__21.5_bf16_clip2.5_LR2e-4_CFG_Bugfix/checkpoints/magpie-tts--val_loss=10.8477-epoch=228-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__21.5_bf16_clip2.5_LR2e-4_CFG_Bugfix/hparams.yaml"

# DCCE 1x: shows special token issue
#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_1x_MG_768_4_12/checkpoints/magpie-tts--val_loss=10.0863-epoch=198-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_1x_MG_768_4_12/hparams.yaml"

# fp32
checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/exps_fixed_causal/dc_21.5_halfcausal_MG_fp32/mapgie-tts-maskgit/0/checkpoints/mapgie-tts-maskgit--val_loss=10.2224-epoch=183-last.ckpt"
hparams_file="/datap/misc/continuouscheckpoints/maskgit/exps_fixed_causal/dc_21.5_halfcausal_MG_fp32/mapgie-tts-maskgit/0/hparams.yaml"

# decoder_context, 2x: shows special tokens issue
#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__21.5_bf16_clip2.5_LR2e-4_CFG_heads4/checkpoints/magpie-tts--val_loss=10.8582-epoch=151-last.ckpt" #"/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__21.5_bf16_clip2.5_LR2e-4_CFG_Bugfix/checkpoints/magpie-tts--val_loss=10.8477-epoch=228-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__21.5_bf16_clip2.5_LR2e-4_CFG_heads4/hparams.yaml" #"/datap/misc/continuouscheckpoints/maskgit/2x/downsample_2x_MG__21.5_bf16_clip2.5_LR2e-4_CFG_Bugfix/hparams.yaml"

# DCCE 2x downsampled: shows special tokens issue
#checkpoint_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_2x_MG_768_4_12/checkpoints/magpie-tts--val_loss=10.6772-epoch=231-last.ckpt"
#hparams_file="/datap/misc/continuouscheckpoints/maskgit/2x/dcce_downsample_2x_MG_768_4_12/hparams.yaml"

codecmodel_path="/datap/misc/checkpoints/ml-model-INTERSPEECH_2025_abblations_21.5Hz_8_codebooks_2016_codes_enc_non_causal_dec_causal_1.89kbps.nemo"



# Temp out dir for saving audios
out_dir = "/datap/misc/t5tts_inference_notebook_samples"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

### Load Model

In [ ]:
hparams_file_from_wandb = False
model_cfg = OmegaConf.load(hparams_file)
if hparams_file_from_wandb:
    model_cfg = model_cfg.cfg.value
else:
    model_cfg = model_cfg.cfg


with open_dict(model_cfg):
    model_cfg.codecmodel_path = codecmodel_path
    if hasattr(model_cfg, 'text_tokenizer'):
        # Backward compatibility for models trained with absolute paths in text_tokenizer
        model_cfg.text_tokenizer.g2p.phoneme_dict = "scripts/tts_dataset_files/ipa_cmudict-0.7b_nv23.01.txt"
        model_cfg.text_tokenizer.g2p.heteronyms = "scripts/tts_dataset_files/heteronyms-052722"
        model_cfg.text_tokenizer.g2p.phoneme_probability = 1.0
    model_cfg.train_ds = None
    model_cfg.validation_ds = None


model = magpietts.MagpieTTSModel(cfg=model_cfg)
print("Loading weights from checkpoint")
ckpt = torch.load(checkpoint_file, weights_only=False)
model.load_state_dict(ckpt['state_dict'])
print("Loaded weights.")

model.use_kv_cache_for_inference = True

model.cuda()
model.eval()

### Initialize Dataset class and helper functions

In [ ]:
test_dataset = MagpieTTSDataset(
    dataset_meta={},
    sample_rate=model.sample_rate,
    min_duration=0.5,
    max_duration=20,
    codec_model_samples_per_frame=model.codec_model_samples_per_frame,
    bos_id=model.bos_id,
    eos_id=model.eos_id,
    context_audio_bos_id=model.context_audio_bos_id,
    context_audio_eos_id=model.context_audio_eos_id,
    audio_bos_id=model.audio_bos_id,
    audio_eos_id=model.audio_eos_id,
    num_audio_codebooks=model.num_audio_codebooks,
    prior_scaling_factor=None,
    load_cached_codes_if_available=True,
    dataset_type='test',
    tokenizer_config=None,
    load_16khz_audio=model.model_type == 'single_encoder_sv_tts',
    use_text_conditioning_tokenizer=model.use_text_conditioning_encoder,
    pad_context_text_to_max_duration=model.pad_context_text_to_max_duration,
    context_duration_min=model.cfg.get('context_duration_min', 5.0),
    context_duration_max=model.cfg.get('context_duration_max', 5.0),
)
test_dataset.text_tokenizer, test_dataset.text_conditioning_tokenizer = setup_tokenizers(
           model_cfg.text_tokenizers, 
           model_cfg.use_text_conditioning_encoder,
           mode='test')


def get_audio_duration(file_path):
    with sf.SoundFile(file_path) as audio_file:
        # Calculate the duration
        duration = len(audio_file) / audio_file.samplerate
        return duration

def create_record(text, context_audio_filepath=None, context_text=None):
    dummy_audio_fp = os.path.join(out_dir, "dummy_audio.wav")
    dummy_audio = sf.write(dummy_audio_fp, np.zeros(22050 * 3), 22050)  # 3 seconds of silence
    record = {
        'audio_filepath' : dummy_audio_fp,
        'duration': 3.0,
        'text': text,
        'speaker': "dummy",
    }
    if context_text is not None:
        assert context_audio_filepath is None
        record['context_text'] = context_text
    else:
        assert context_audio_filepath is not None
        record['context_audio_filepath'] = context_audio_filepath
        record['context_audio_duration'] = get_audio_duration(context_audio_filepath)
    
    return record

### Set transcript and context pairs to test

In [ ]:
import pprint
# Change sample text and prompt audio/text here
audio_base_dir = "/"
test_entries = [
    create_record(
        text="The recollection of a unique event cannot, so Bergson contends, be wholly constituted by habit, and is in fact something radically different from the memory which is habit.",
        #text="First you called it T-five-Tee-tee-eS, then Koel-Tee-Tee-es, now Magpie... Make up your minds, folks!",
        #text="First you called it T five T T S. Then Koel. Now, Magpie... Make up your mind, folks!",
#         text="This is a simple sentence to check my text to speech synthesis model.",
#         text=" How? ",
#         context_text="Speaker and Emotion: | Language:en Dataset:Riva Speaker:Lindy_WIZWIKI |",
#         Lindy_CMU_FEARFUL
#         Lindy_WIZWIKI
#         context_audio_filepath="/datap/misc/LibriTTSfromNemo/LibriTTS/test-clean/7729/102255/7729_102255_000012_000001.wav", # Supply either context_audio_filepath or context_text, not both
        #context_audio_filepath="/datap/misc/Datasets/LibriTTS/test-clean/8230/279154/8230_279154_000004_000009.wav",
        context_audio_filepath="/datap/misc/Datasets/LibriTTS/test-clean/7176/88083/7176_88083_000018_000001.wav",
        #context_audio_filepath="/home/rfejgin/contexts/boris-snippet-dereverb-denoise-short.wav"
    ),
]

data_samples = []
for entry in test_entries:
    dataset_sample = DatasetSample(
        dataset_name="sample",
        manifest_entry=entry,
        audio_dir=audio_base_dir,
        feature_dir=audio_base_dir,
        text=entry['text'],
        speaker=None,
        speaker_index=0,
        tokenizer_names=["english_phoneme"], # Change this for multilingual: "english_phoneme", "spanish_phoneme", "english_chartokenizer", "german_chartokenizer".. 
    )
    data_samples.append(dataset_sample)
    
test_dataset.data_samples = data_samples

test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    collate_fn=test_dataset.collate_fn,
    num_workers=0,
    shuffle=False
)

### Generate With Prior

In [ ]:
import matplotlib.pyplot as plt

item_idx = 0
for bidx, batch in enumerate(test_data_loader):
    print("Processing batch {} out of {}".format(bidx, len(test_data_loader)))
    model.decoder.reset_cache(use_cache=True)
    batch_cuda ={}
    for key in batch:
        if isinstance(batch[key], torch.Tensor):
            batch_cuda[key] = batch[key].cuda()
        else:
            batch_cuda[key] = batch[key]
    import time
    st = time.time()
    maskgit_n_steps = 8
    fixed_schedule_n_unmasked = None
    #fixed_schedule_n_unmasked = [0,1,4,8,9,12] #[0, 2, 8, 9, 12]#[0,1,2,8,9,11]#[0,1,4,8,9]#[0,1,2,4,10] #[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] #[0, 1, 2, 5, 8]# [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]# [0, 1, 2, 5, 8]# 4, 8 ]
    #fixed_schedule_n_unmasked = #[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]#[0,1,2,8,9]#[0,1,2,3,8,9,10,11] #[0,1,2,4,8,11]#[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    #fixed_schedule_n_unmasked = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31] # [0,1,2,8,9,10,16,17,18,24,25,26]
    #fixed_schedule_n_unmasked = [0,2,8,10,16,18,24,26] # [0,2,4,8,10,12,16,18,20,24,26,28]
    #fixed_schedule_n_unmasked =  [0,1,8,9]#]#[0,1,16]#][0,1,8,9,16,17,24,25]# [0,1,16]#][0,1,8,9,16,17,24,25]#[0,1,8,16,24]#[0,1,8,9,16,24]#[0,1,8,16]#[0,1,8,9,16,17,24,25]#[0,1,2,8,9,10,16,17,18,24,25,26]# [0,1,2, 8,9,10, 16,17,18]# [0,1,8,9,16,17]
    for _ in range(4):
        for sampling_type in ["purity_causal", "purity_default", "causal"]:# ["causal"]: #["default", "causal"]:
            for use_local_transformer_for_inference in [True]:#[True]:#[False, True]:
                for i, maskgit_noise_scale in enumerate([0]):#,0.1,1]):#[0, 0.01, 0.1, 1]):#[0,1e-3]):#([0.0, 1e-3, 1e-2, 5e-2, 1e-1]):
                    if not use_local_transformer_for_inference and i > 0:
                        continue
                    for apply_prior in [False]:
                        predicted_audio, predicted_audio_lens, _, _, rtf_metrics, cross_attn_np, all_heads_attn_np = model.infer_batch(
                            batch_cuda, 
                            max_decoder_steps=430, 
                            temperature=0.7,#,0.55,#5,#0.7,#0.6, 
                            topk=80, 
                            use_cfg=False,#True,
                            cfg_scale=2.5,#2.5,#3.0,#2.5,#4.5,#3.5,#5.5,#3.5,#8.0,#3.5,#2.5,#4.5,#2.5,#2.5,
                            return_cross_attn_probs=True,
                            apply_attention_prior=False,
                            compute_all_heads_attn_maps=True,
                            use_local_transformer_for_inference=use_local_transformer_for_inference,
                            maskgit_n_steps=maskgit_n_steps,
                            maskgit_noise_scale=0,#0.1,#maskgit_noise_scale,
                            fixed_schedule_n_unmasked=fixed_schedule_n_unmasked,
                            dynamic_cfg_scale=False,
                            sampling_type=sampling_type
                        )
                        print("generation time", time.time() - st)
                        pprint.pprint(rtf_metrics)
                        for idx in range(predicted_audio.size(0)):
                            predicted_audio_np = predicted_audio[idx].float().detach().cpu().numpy()
                            predicted_audio_np = predicted_audio_np[:predicted_audio_lens[idx]]
                            audio_path = os.path.join(out_dir, f"predicted_audio_{item_idx}.wav")
                            sf.write(audio_path, predicted_audio_np, model.sample_rate)
                            print(f"i={i}")
                            print(test_entries[bidx]['text'])
                            print("Prior Used? ", apply_prior)
                            print("use_local_transformer_for_inference? ", use_local_transformer_for_inference)
                            print("maskgit_n_steps: ", maskgit_n_steps)
                            print("maskgit_noise_scale: ", maskgit_noise_scale)
                            print("sampling_type: ", sampling_type)
                            print("checkpoint: ", checkpoint_file)
                            display(Audio(audio_path))
                            item_idx += 1

                            print("------------------------------------")
                            print("------------------------------------")

In [ ]:
checkpoint_file